In [268]:
import requests
import os
import yaml
import pandas as pd
import numpy as np
import re
import pickle
from keras.models import model_from_json
from sklearn.externals import joblib
import json

- Import credentials

In [217]:
credentials = yaml.load(open(os.path.expanduser('~/.ssh/lending_club_api.yml')))

In [218]:
credentials

{'account_id': 71351615, 'authentication': 'lOYMBaOCpJorkX62/RCOZ1Vezn0='}

In [219]:
account_id = credentials['account_id']

In [220]:
account_id

71351615

- Make the api call for the portfolios owned
    - Find the API portfolio to put loans into

In [221]:
portfolio_r = requests.get("https://api.lendingclub.com/api/investor/v1/accounts/{}/portfolios".format(account_id),
                 headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                          'Accept': 'application/json'}, params = {'showAll':'true'})

In [222]:
portfolio_id = 0
for portfolio in portfolio_r.json()['myPortfolios']:
    if portfolio['portfolioName']=='api': # Put api orders in this portfolio
        portfolio_id = portfolio['portfolioId']
    

- Make the API call to see the loans listed

In [223]:

r = requests.get('https://api.lendingclub.com/api/investor/v1/loans/listing',
                 headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                          'Accept': 'application/json'}, params = {'showAll':'true'})

In [224]:
r

<Response [200]>

In [225]:
json_response =  r.json()

In [226]:
json_response.keys()

[u'loans', u'asOfDate']

In [227]:
test_loan = json_response['loans'][50]

In [228]:
test_loan['purpose']

u'debt_consolidation'

- Convert to DF

In [229]:
loans_df = pd.DataFrame(json_response['loans'])

In [230]:
loans_df.head()

,accNowDelinq,accOpenPast24Mths,acceptD,addrState,addrZip,allUtil,annualInc,annualIncJoint,applicationType,avgCurBal,...,totCollAmt,totCurBal,totHiCredLim,totalAcc,totalBalExMort,totalBalIl,totalBcLimit,totalCuTl,totalIlHighCreditLimit,totalRevHiLim
0,0,2,2017-09-30T17:46:38.000-07:00,MA,021xx,21.4,49000.0,NaN,INDIVIDUAL,2697,...,0,16181,75850,17,16181,15357.0,51300,0,24550,51300
1,0,3,2017-10-03T13:04:40.000-07:00,NH,038xx,77.8,152028.0,NaN,INDIVIDUAL,63996,...,0,511971,527824,21,23664,0.0,23900,0,0,30400
2,0,3,2017-09-19T16:01:49.000-07:00,IL,622xx,53.3,45000.0,NaN,INDIVIDUAL,3049,...,0,24391,45777,13,24391,3198.0,19000,2,16077,29700
3,0,4,2017-10-04T07:15:43.000-07:00,VA,234xx,74.9,33000.0,145000.0,JOINT,9592,...,0,67142,90629,21,67142,52794.0,2800,9,73529,17100
4,0,1,2017-09-15T20:40:15.000-07:00,CA,960xx,78.3,70000.0,NaN,INDIVIDUAL,44958,...,0,314705,356571,9,50439,26040.0,30100,0,34322,30100


In [231]:
loans_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 103 columns):
accNowDelinq                  int64
accOpenPast24Mths             int64
acceptD                       object
addrState                     object
addrZip                       object
allUtil                       float64
annualInc                     float64
annualIncJoint                float64
applicationType               object
avgCurBal                     int64
bcOpenToBuy                   float64
bcUtil                        float64
chargeoffWithin12Mths         int64
collections12MthsExMed        int64
creditPullD                   object
delinq2Yrs                    int64
delinqAmnt                    float64
desc                          object
dti                           float64
dtiJoint                      float64
earliestCrLine                object
empLength                     float64
empTitle                      object
expD                          object
exp

In [232]:
loans_df.id.head()

0    120158005
1    120215482
2    119304363
3    120234827
4    118945408
Name: id, dtype: int64

#### Convert EmpLength to years

In [233]:
loans_df.empLength = loans_df.empLength.apply(lambda x: x/60 if x >0 else 0)

#### Create month to int 1-12

In [234]:
loans_df['month'] = loans_df.acceptD.apply(lambda x: pd.to_datetime(x).month)

#### Crea cols term_36 and term_60

In [235]:
loans_df['term_36'] = [1 if i ==36 else 0 for i in loans_df.term]

In [236]:
loans_df['term_60'] = [1 if i ==60 else 0 for i in loans_df.term]

#### Create grade columns
- a:g

In [237]:
loans_df['grade_a'] = [1 if i =='A' else 0 for i in loans_df.grade]
loans_df['grade_b']= [1 if i =='B' else 0 for i in loans_df.grade]
loans_df['grade_c']= [1 if i =='C' else 0 for i in loans_df.grade]
loans_df['grade_d']= [1 if i =='D' else 0 for i in loans_df.grade]
loans_df['grade_e']= [1 if i =='E' else 0 for i in loans_df.grade]
loans_df['grade_f']= [1 if i =='F' else 0 for i in loans_df.grade]
loans_df['grade_g']= [1 if i =='G' else 0 for i in loans_df.grade]

### Create home ownership columns
-'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',

In [238]:
loans_df['home_ownership_any'] = [1 if i =='ANY' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_mortgage'] = [1 if i =='MORTGAGE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_none'] = [1 if i =='NONE' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_other'] = [1 if i =='OTHER' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_own'] = [1 if i =='OWN' else 0 for i in loans_df.homeOwnership]
loans_df['home_ownership_rent'] = [1 if i =='RENT' else 0 for i in loans_df.homeOwnership]

## Create purpose columns
-'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',

In [239]:
loans_df['purpose_credit_card'] = [1 if i =='credit_card' else 0 for i in loans_df.purpose]
loans_df['purpose_debt_consolidation'] = [1 if i =='debt_consolidation' else 0 for i in loans_df.purpose]
loans_df['purpose_educational'] = [1 if i =='educational' else 0 for i in loans_df.purpose]
loans_df['purpose_home_improvement'] = [1 if i =='home_improvement' else 0 for i in loans_df.purpose]
loans_df['purpose_house'] = [1 if i =='house' else 0 for i in loans_df.purpose]
loans_df['purpose_major_purchase'] = [1 if i =='major_purchase' else 0 for i in loans_df.purpose]
loans_df['purpose_medical'] = [1 if i =='medical' else 0 for i in loans_df.purpose]
loans_df['purpose_moving'] = [1 if i =='moving' else 0 for i in loans_df.purpose]
loans_df['purpose_other'] = [1 if i =='other' else 0 for i in loans_df.purpose]
loans_df['purpose_renewable_energy'] = [1 if i =='renewable_energy' else 0 for i in loans_df.purpose]
loans_df['purpose_small_business'] = [1 if i =='small_business' else 0 for i in loans_df.purpose]
loans_df['purpose_vacation'] = [1 if i =='vacation' else 0 for i in loans_df.purpose]
loans_df['purpose_wedding'] = [1 if i =='wedding' else 0 for i in loans_df.purpose]
loans_df['purpose_car'] = [1 if i =='car' else 0 for i in loans_df.purpose]




### Create the states columns
'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY'

In [240]:
loans_df['addr_state_AK'] = [1 if i =='AK' else 0 for i in loans_df.addrState]
loans_df['addr_state_AL'] = [1 if i =='AL' else 0 for i in loans_df.addrState]
loans_df['addr_state_AR'] = [1 if i =='AR' else 0 for i in loans_df.addrState]
loans_df['addr_state_AZ'] = [1 if i =='AZ' else 0 for i in loans_df.addrState]
loans_df['addr_state_CA'] = [1 if i =='CA' else 0 for i in loans_df.addrState]
loans_df['addr_state_CO'] = [1 if i =='CO' else 0 for i in loans_df.addrState]
loans_df['addr_state_CT'] = [1 if i =='CT' else 0 for i in loans_df.addrState]
loans_df['addr_state_DC'] = [1 if i =='DC' else 0 for i in loans_df.addrState]
loans_df['addr_state_DE'] = [1 if i =='DE' else 0 for i in loans_df.addrState]
loans_df['addr_state_FL'] = [1 if i =='FL' else 0 for i in loans_df.addrState]
loans_df['addr_state_GA'] = [1 if i =='GA' else 0 for i in loans_df.addrState]
loans_df['addr_state_HI'] = [1 if i =='HI' else 0 for i in loans_df.addrState]
loans_df['addr_state_IA'] = [1 if i =='IA' else 0 for i in loans_df.addrState]
loans_df['addr_state_ID'] = [1 if i =='ID' else 0 for i in loans_df.addrState]
loans_df['addr_state_IL'] = [1 if i =='IL' else 0 for i in loans_df.addrState]
loans_df['addr_state_IN'] = [1 if i =='IN' else 0 for i in loans_df.addrState]
loans_df['addr_state_KS'] = [1 if i =='KS' else 0 for i in loans_df.addrState]
loans_df['addr_state_KY'] = [1 if i =='KY' else 0 for i in loans_df.addrState]
loans_df['addr_state_LA'] = [1 if i =='LA' else 0 for i in loans_df.addrState]
loans_df['addr_state_MA'] = [1 if i =='MA' else 0 for i in loans_df.addrState]
loans_df['addr_state_MD'] = [1 if i =='MD' else 0 for i in loans_df.addrState]
loans_df['addr_state_ME'] = [1 if i =='ME' else 0 for i in loans_df.addrState]
loans_df['addr_state_MI'] = [1 if i =='MI' else 0 for i in loans_df.addrState]
loans_df['addr_state_MN'] = [1 if i =='MN' else 0 for i in loans_df.addrState]
loans_df['addr_state_MO'] = [1 if i =='MO' else 0 for i in loans_df.addrState]
loans_df['addr_state_MS'] = [1 if i =='MS' else 0 for i in loans_df.addrState]
loans_df['addr_state_MT'] = [1 if i =='MT' else 0 for i in loans_df.addrState]
loans_df['addr_state_NC'] = [1 if i =='NC' else 0 for i in loans_df.addrState]
loans_df['addr_state_ND'] = [1 if i =='ND' else 0 for i in loans_df.addrState]
loans_df['addr_state_NE'] = [1 if i =='NE' else 0 for i in loans_df.addrState]
loans_df['addr_state_NH'] = [1 if i =='NH' else 0 for i in loans_df.addrState]
loans_df['addr_state_NJ'] = [1 if i =='NJ' else 0 for i in loans_df.addrState]
loans_df['addr_state_NM'] = [1 if i =='NM' else 0 for i in loans_df.addrState]
loans_df['addr_state_NV'] = [1 if i =='NV' else 0 for i in loans_df.addrState]
loans_df['addr_state_NY'] = [1 if i =='NY' else 0 for i in loans_df.addrState]
loans_df['addr_state_OH'] = [1 if i =='OH' else 0 for i in loans_df.addrState]
loans_df['addr_state_OK'] = [1 if i =='OK' else 0 for i in loans_df.addrState]
loans_df['addr_state_OR'] = [1 if i =='OR' else 0 for i in loans_df.addrState]
loans_df['addr_state_PA'] = [1 if i =='PA' else 0 for i in loans_df.addrState]
loans_df['addr_state_RI'] = [1 if i =='RI' else 0 for i in loans_df.addrState]
loans_df['addr_state_SC'] = [1 if i =='SC' else 0 for i in loans_df.addrState]
loans_df['addr_state_SD'] = [1 if i =='SD' else 0 for i in loans_df.addrState]
loans_df['addr_state_TN'] = [1 if i =='TN' else 0 for i in loans_df.addrState]
loans_df['addr_state_TX'] = [1 if i =='TX' else 0 for i in loans_df.addrState]
loans_df['addr_state_UT'] = [1 if i =='UT' else 0 for i in loans_df.addrState]
loans_df['addr_state_VA'] = [1 if i =='VA' else 0 for i in loans_df.addrState]
loans_df['addr_state_VT'] = [1 if i =='VT' else 0 for i in loans_df.addrState]
loans_df['addr_state_WA'] = [1 if i =='WA' else 0 for i in loans_df.addrState]
loans_df['addr_state_WI'] = [1 if i =='WI' else 0 for i in loans_df.addrState]
loans_df['addr_state_WV'] = [1 if i =='WV' else 0 for i in loans_df.addrState]
loans_df['addr_state_WY'] = [1 if i =='WY' else 0 for i in loans_df.addrState]



#### Create the zip code column

In [241]:
int(loans_df.addrZip[0][:3])

21

- Drop null zip codes

In [242]:
loans_df.addrZip = loans_df.addrZip.apply( lambda x: float(x[:3]))

In [243]:
loans_df.columns

Index([     u'accNowDelinq', u'accOpenPast24Mths',           u'acceptD',
               u'addrState',           u'addrZip',           u'allUtil',
               u'annualInc',    u'annualIncJoint',   u'applicationType',
               u'avgCurBal',
       ...
           u'addr_state_SD',     u'addr_state_TN',     u'addr_state_TX',
           u'addr_state_UT',     u'addr_state_VA',     u'addr_state_VT',
           u'addr_state_WA',     u'addr_state_WI',     u'addr_state_WV',
           u'addr_state_WY'],
      dtype='object', length=184)

In [244]:
# Columns to match the data model
# empLength is in months - convert to years
api_cols =['loanAmount',
           'fundedAmount',
           'intRate',
           'installment',
           'empLength',
           'annualInc',
           'addrZip',
          'dti',
           'delinq2Yrs',
           'inqLast6Mths',
       'mthsSinceLastDelinq',
           'mthsSinceLastRecord',
           'openAcc',
           'pubRec',
           'revolBal',
           'revolUtil',
           'totalAcc',
      'collections12MthsExMed',
           'mthsSinceLastMajorDerog',
           'accNowDelinq',
           'totCollAmt',
           'totCurBal',
           'openAcc6m',
      'openIl6m',
        'openIl12m',
           'openIl24m',
           'totalBalIl',
           'iLUtil',
           'maxBalBc',
           'allUtil',
        'totalRevHiLim',
           'inqLast12m',
           'accOpenPast24Mths',
      'avgCurBal',
           'bcUtil',
           'chargeoffWithin12Mths',
           'delinqAmnt',
           'moSinOldIlAcct',
           'moSinOldRevTlOp',
           'moSinRcntRevTlOp',
     'moSinRcntTl',
           'mortAcc',
           'mthsSinceRecentBc',
           'mthsSinceRecentBcDlq',
           'mthsSinceRecentInq',
           'mthsSinceRecentRevolDelinq',
      'numAcctsEver120Ppd',
           'numActvBcTl',
           'numActvRevTl',
           'numBcSats',
           'numBcTl',
           'numIlTl',
           'numOpRevTl',
           'numRevAccts',
      'numRevTlBalGt0',
           'numSats',
           'numTl120dpd2m',
           'numTl30dpd',
           'numTl90gDpd24m',
           'numTlOpPast12m',
           'pctTlNvrDlq',
      'percentBcGt75',
           'pubRecBankruptcies',
           'taxLiens',
           'totHiCredLim',
           'totalBalExMort',
           'totalBcLimit',
           'totalIlHighCreditLimit',
      'month',
           'term_36',
           'term_60',
           'grade_a',
           'grade_b',
           'grade_c',
           'grade_d',
           'grade_e',
           'grade_f',
           'grade_g',
      'home_ownership_any','home_ownership_mortgage','home_ownership_none','home_ownership_other','home_ownership_own',
      'home_ownership_rent',
           'purpose_car','purpose_credit_card','purpose_debt_consolidation','purpose_educational','purpose_home_improvement',
      'purpose_house','purpose_major_purchase','purpose_medical','purpose_moving','purpose_other','purpose_renewable_energy',
      'purpose_small_business','purpose_vacation','purpose_wedding',
           'addr_state_AK','addr_state_AL','addr_state_AR',
      'addr_state_AZ','addr_state_CA','addr_state_CO','addr_state_CT','addr_state_DC','addr_state_DE','addr_state_FL',
      'addr_state_GA','addr_state_HI','addr_state_IA','addr_state_ID','addr_state_IL','addr_state_IN','addr_state_KS',
      'addr_state_KY','addr_state_LA','addr_state_MA','addr_state_MD','addr_state_ME','addr_state_MI','addr_state_MN',
      'addr_state_MO','addr_state_MS','addr_state_MT','addr_state_NC','addr_state_ND','addr_state_NE','addr_state_NH',
      'addr_state_NJ','addr_state_NM','addr_state_NV','addr_state_NY','addr_state_OH','addr_state_OK','addr_state_OR',
      'addr_state_PA','addr_state_RI','addr_state_SC','addr_state_SD','addr_state_TN','addr_state_TX','addr_state_UT',
      'addr_state_VA','addr_state_VT','addr_state_WA','addr_state_WI','addr_state_WV','addr_state_WY']



In [245]:
len(api_cols)

149

### Create the final data frame

In [246]:
final_api_df = loans_df[api_cols]

# Data Model
#### This is what the model was trained on
```'loan_amnt',
 'funded_amnt',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'zip_code',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'month',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY'] ```

In [247]:
total_x_cols = ['loan_amnt',
 'funded_amnt',
 'int_rate',
 'installment',
 'emp_length',
 'annual_inc',
 'zip_code',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_inq',
 'mths_since_recent_revol_delinq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'month',
 'term_ 36 months',
 'term_ 60 months',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'home_ownership_ANY',
 'home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_car',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'addr_state_AK',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY']

In [248]:
y_col = ['repaid']

In [249]:
len(total_x_cols)

149

In [250]:
len(api_cols)

149

- Same number of columns and in the correct order 


### Impute zeros for NaNs

In [251]:
final_api_df.fillna(0,inplace=True)

In [252]:
final_api_df.iloc[:,:20] .head()

,loanAmount,fundedAmount,intRate,installment,empLength,annualInc,addrZip,dti,delinq2Yrs,inqLast6Mths,mthsSinceLastDelinq,mthsSinceLastRecord,openAcc,pubRec,revolBal,revolUtil,totalAcc,collections12MthsExMed,mthsSinceLastMajorDerog,accNowDelinq
0,6025.0,5975.0,10.42,195.61,0.0,49000.0,21.0,14.43,2,0,9.0,82.0,7,3,824.0,1.5,17,0,25.0,0
1,20000.0,19950.0,9.93,644.69,1.2,152028.0,38.0,6.36,1,0,23.0,0.0,9,0,23664.0,77.8,21,0,67.0,0
2,20000.0,17950.0,30.79,656.81,1.2,45000.0,622.0,25.17,0,1,81.0,0.0,9,0,21193.0,71.4,13,0,0.0,0
3,1600.0,1525.0,15.05,55.51,0.0,33000.0,234.0,65.78,0,2,48.0,0.0,7,0,14348.0,83.9,21,0,0.0,0
4,32000.0,15000.0,9.44,1024.16,0.8,70000.0,960.0,22.48,0,0,0.0,113.0,7,1,24399.0,81.1,9,0,0.0,0


In [253]:
final_api_df.iloc[:,20:40] .head()

,totCollAmt,totCurBal,openAcc6m,openIl6m,openIl12m,openIl24m,totalBalIl,iLUtil,maxBalBc,allUtil,totalRevHiLim,inqLast12m,accOpenPast24Mths,avgCurBal,bcUtil,chargeoffWithin12Mths,delinqAmnt,moSinOldIlAcct,moSinOldRevTlOp,moSinRcntRevTlOp
0,0,16181,1,1,0,1,15357.0,62.6,303.0,21.4,51300,0,2,2697,1.5,0,0.0,146.0,205,0
1,0,511971,1,0,0,0,0.0,0.0,19706.0,77.8,30400,0,3,63996,93.7,0,0.0,148.0,216,3
2,0,24391,2,1,0,0,3198.0,19.9,5067.0,53.3,29700,1,3,3049,86.1,0,0.0,142.0,293,2
3,0,67142,2,2,0,2,52794.0,70.4,1331.0,74.9,17100,3,4,9592,47.5,0,0.0,124.0,241,3
4,0,314705,0,1,0,1,26040.0,75.9,10671.0,78.3,30100,0,1,44958,81.1,0,0.0,21.0,189,29


# Load the trained models

#### Gradient Boosting

In [279]:
# load the model from disk
loaded_gb_model = joblib.load(open("./models/gradient_boosting_400-trees.joblib.pkl", 'rb'))

In [281]:
loaded_gb_model.predict(final_api_df)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [282]:
loaded_gb_model.predict(final_api_df).sum()

0

In [283]:
loans_to_purchase = loans_df.id[loaded_gb_model.predict(final_api_df)==1] #find the indexes of where loans predicted to r
# repay = 1

#### Keras

In [141]:
#Taken from https://machinelearningmastery.com/save-load-keras-deep-learning-models/
# load json and create model
json_file = open('./models/mlp_model_5-epochs_64-batchsize_0.761329171758-train_acc_0.762269526-val_acc', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./models/mlp_model_5-epochs_64-batchsize_0.761758990161-train_acc_0.763773047407-val_acc.h5")
print("Loaded model from disk")

Loaded model from disk


# Check available cash
```
URL: https://api.lendingclub.com/api/investor/<version>/accounts/<investor id>/availablecash
```

In [272]:

cash_r = requests.get('https://api.lendingclub.com/api/investor/v1/accounts/{}/availablecash'.format(account_id),
                 headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                          'Accept': 'application/json'}, params = {'showAll':'true'})

In [273]:
available_cash = cash_r.json()['availableCash'] # see if available cash > $25

In [274]:
available_cash

27.68

# Create orders
```

Operation: POST
URL: https://api.lendingclub.com/api/investor/<version>/accounts/<investor id>/orders


{
	"aid":11111,
	"orders":[
	{
		"loanId":22222,
		"requestedAmount":55.0,
		"portfolioId":44444
	},
	{
		"loanId": 33333,
		"requestedAmount":25,
		"portfolioId":55555
	},
	{
		"loanId": 44444,
		"requestedAmount":25,
	}]
}```

In [204]:
num_orders = available_cash // 25 ## total number of possible orders of $25

In [270]:
def purchase_loans(available_cash,account_id,loans_to_purchase, portfolio_id):
    """"""
count = 0
while count != len(loans_to_purchase):
    if (len(loans_to_purchase) >0) & (available_cash > 25): # buy some loans!

 
        payload = json.dumps({'aid': account_id, \
                   'orders':[{'loanId' : loans_to_purchase[count], \
                                'requestedAmount' : float(25), \
                                'portfolioId' : portfolio_id}]})



        buy_loans_r = requests.post('https://api.lendingclub.com/api/investor/v1/accounts/{}/orders'.format(account_id),
                                    data=payload,
                     headers={'Authorization': credentials['authentication'],'Content-Type': 'application/json',
                              'Accept': 'application/json'}, params = {'showAll':'true'})
        count +=1
    else:
        print('Nothing to purchase')
        break
    

In [271]:
 buy_loans_r.text

u'{"orderInstructId":168357062,"orderConfirmations":[{"loanId":120158005,"requestedAmount":25.0,"investedAmount":25.0,"executionStatus":["ORDER_FULFILLED","NOTE_ADDED_TO_PORTFOLIO"]}]}'

>>> payload = {'key1': 'value1', 'key2': 'value2'}

>>> r = requests.post("http://httpbin.org/post", data=payload)
payload = (('key1', 'value1'), ('key1', 'value2'))
>>> print(r.text)
{

In [ ]:
portfolio_id , account_id